# Part 1: Create environment and define functions

In [ ]:
import pandas as pd
import numpy as np
import os
from arcgis.features import SpatialDataFrame

# no need to add intermediate inputs to map
arcpy.env.addOutputsToMap = 0

In [ ]:
# function that collects and processes the outputs of the sub-functions defined below
def main(start, end, populations, out_fc, out_csv):
    
    # get the number of trips accessible to each block at the start and end of the study period
    start_trips = blocks_trips_count (start['demo'], 
                                      start['gtfs'], 
                                      start['agency_ids'], 
                                      start['service_ids'], 
                                      'start')
    
    end_trips = blocks_trips_count (end['demo'], 
                                    end['gtfs'], 
                                    end['agency_ids'], 
                                    end['service_ids'], 
                                    'end')
    
    # create start and end tables with fields for trip counts, selected population counts, and a join field for each block
    create_table('start', start_trips, populations, start['join_on'])
    create_table('end', end_trips, populations, end['join_on'])
   
    
    # join the tables together
    joined_table = arcpy.management.JoinField(r'memory\start_table', 
                                              'start_'+start['join_on'], 
                                              r'memory\end_table', 
                                              'end_'+end['join_on'])
    
    # end_table no longer needed, since its values were joined to start_table
    arcpy.management.Delete(r'memory\end_table') 
    
    # calculate the weighted service level change for the each sub-population
    # append block-level changes to service_change_fc and calculate summaries for each subgroup
    
    output_array = []  # initialize array to hold total service level changes for each population
    pop_index = 0
    for population in populations:
        fieldNames = ['start_trip_count', 
                      'end_trip_count', 
                      'start_'+populations[population][0], 
                      'end_'+populations[population][1]
                     ]
        output_array.append(service_change(fieldNames, population, pop_index, joined_table))
        pop_index += 1
        
    # print status message
    print('Writing Output')

    # create a new feature class using the geometry of the start demography to hold the block level service change for each population subgroup
    fms = arcpy.FieldMappings()
    fm = arcpy.FieldMap()
    fm.addInputField(start['demo'], start['join_on'])
    fms.addFieldMap(fm)
    service_change_fc = arcpy.conversion.FeatureClassToFeatureClass(start['demo'], workspace, out_fc, field_mapping=fms)
    
    #join the service change data to the service_change feature class geometry
    arcpy.management.JoinField(service_change_fc, 
                               start['join_on'], 
                               r'memory\start_table',
                               'start_'+start['join_on'],
                               ['service_change'+str(i) for i in range(len(populations))])
    
    # start_table no longer needed, since relevant data are attached to the service change fc
    arcpy.management.Delete(r'memory\start_table') 
    
    # Add service change feature class to the active map in the current project
    aprx = arcpy.mp.ArcGISProject('CURRENT')
    aprx.activeMap.addDataFromPath(os.path.join(workspace, out_fc))

    # output summary of service level change to dataframe
    df = pd.DataFrame(output_array, columns=['population', 'service change'])
    
    # export summary table to csv & add to map
    df.to_csv(out_csv, index=False)
    aprx.activeMap.addDataFromPath(out_csv)
    
    return df

In [ ]:
# calculate the number of trips that come within 1/4 mile of each block center
def blocks_trips_count (demo, gtfs, agency_ids, service_ids, time):
    
    # print status message
    print('Counting trips')
    
    # get the total number of trips on each route segment
    
    # set input and output file paths
    trips_path = os.path.join(gtfs, 'trips.txt')
    routes_path = os.path.join(gtfs, 'routes.txt')
    shapes_path = os.path.join(gtfs, 'shapes.txt')
    shapes_trips_path = os.path.join(gtfs, 'shapes_trips.txt')
    shapes_fc_path = r'memory\shapes'
    demo_fc_path = os.path.join(workspace, demo)
    demo_centers_path = r'memory\demo_centers'
    demo_centers_shapes_join_path = r'memory\block_trips_'+time

    # read GTFS data into pandas dataframes
    trips = pd.read_csv(trips_path)
    routes = pd.read_csv(routes_path)

    # filter the trips on selected service ids
    shape_count = trips.query(f'service_id in {service_ids}')
    
    # group selected trips by shape_id to get a count of trips on that shape
    # keep route_id so it can be joined to routes later   
    shape_count = shape_count[['route_id', 'service_id', 'shape_id']].groupby(['shape_id']).agg({'route_id': 'max', 'service_id': 'count'})
    shape_count.rename(index=str, columns={'service_id': 'trip_count'}, inplace=True)

    # join routes info to the trip counts by shape to get the agency_id for each shape
    shape_count['shape_id'] = shape_count.index # need to copy index since merge does not preserve it
    shape_count = shape_count.merge(routes[['route_id', 'agency_id']], on='route_id', how='left')

    # filter by selected agency ids
    shape_count = shape_count.query(f'agency_id in {agency_ids}')

    # create a spatial dataframe of the transit route geometry
    arcpy.conversion.GTFSShapesToFeatures(shapes_path, shapes_fc_path)
    sdf = pd.DataFrame.spatial.from_featureclass(shapes_fc_path)

    # Join the trip counts to the route geometry
    shape_count_geom = sdf.merge(shape_count[['trip_count', 'shape_id']], on='shape_id', how='right')

    # write the sdf back to the shapes feature class
    shape_count_geom.spatial.to_featureclass(shapes_fc_path)

    
    
    # count the total number of trips accessible to each block
    # A stop is accessible to everyone in the block if it is within 1/4 mile of the block center

    # Convert polygon census blocks to points
    arcpy.management.FeatureToPoint(demo_fc_path, 
                                    demo_centers_path, 
                                    "INSIDE")

    # Create the necessary FieldMappings object
    fms = arcpy.FieldMappings()

    # loop through the fields in the census block centers, adding them to the field map
    for field in arcpy.ListFields(demo_centers_path):  
        if field.name != 'OBJECTID' and field.name != 'Shape':  # attempting to add the shape or objectid fields to the field map will throw an exception
            fm1 = arcpy.FieldMap()
            fm1.addInputField(demo_centers_path, field.name)
            fm1.mergeRule = 'First'
            fms.addFieldMap(fm1)

    # Add the trip count field from the shapes fc to a FieldMap object
    fm2 = arcpy.FieldMap()
    fm2.addInputField(shapes_fc_path, "trip_count")

    # we want the total number of trips to join to each block center
    fm2.mergeRule = 'Sum'

    # Add the trip count FieldMap to the FieldMappings Object
    fms.addFieldMap(fm2)

    # Join the route counts to the block centers if they are within 0.25 miles of the center
    arcpy.analysis.SpatialJoin(demo_centers_path, 
                               shapes_fc_path, 
                               demo_centers_shapes_join_path, # output feature class
                               "JOIN_ONE_TO_ONE", 
                               "KEEP_ALL", 
                               field_mapping=fms,
                               match_option="INTERSECT", 
                               search_radius="0.25 Miles"
                              )
    
    # the route shapes and block centers are no longer needed
    arcpy.management.Delete(shapes_fc_path)
    arcpy.management.Delete(demo_centers_path)
    
    return demo_centers_shapes_join_path

In [ ]:
# create a table of just the needed fields for calculating population weighted service level change
def create_table(start_or_end, in_table, populations, join_field):
    
    # print status message
    print('Creating data tables')
    
    # make sure that the start table gets the first value for each key in the populations dict
    # and the end table gets the second, since the field names might be different in the two tables
    out_table_name = start_or_end+'_table'
    if start_or_end == 'start':
        field_list_index = 0
    else: field_list_index = 1
    
    # create a field list starting with the join field and trip counts
    fields = [join_field, 'trip_count']

    # extend the field list to include fields for selected population groups
    fields.extend([populations[field_list][field_list_index] for field_list in populations])

    # iterate through the fields and add them to a field mapping
    fms = arcpy.FieldMappings()
    for field in fields:
        fm = arcpy.FieldMap()
        fm.addInputField(in_table, field)
        
        #change the output field name to differentiate b/w fields in the start and end table, 
        # in case they are the same
        field_name = fm.outputField
        field_name.name = start_or_end+'_'+field
        fm.outputField = field_name        
        
        fms.addFieldMap(fm)

    # create the table
    out_table = arcpy.conversion.TableToTable(in_table, 'memory', out_table_name, field_mapping=fms)
    
    # the block_trips feature class is no longer needed since the relevant fields were written to this table
    arcpy.management.Delete(in_table)
    

In [ ]:
# for a given subgroup population, cacluate both the population weighted service level change for each block,
# as well as the total service level change for the population as a whole
def service_change(fieldNames, population, pop_index, joined_table):
    
    # print status message
    print('Calculating service change for '+population)
    
    # add a field for the service level change for the population subgroup
    service_change_field = 'service_change'+str(pop_index)
    arcpy.management.AddField(joined_table, 
                              service_change_field, 
                              field_type = 'DOUBLE', 
                              field_alias = population+' change')
    
    fieldNames.append(service_change_field)

    
    # iterate through every block to calcuate the service level change for the selected population in that block
    with arcpy.da.UpdateCursor(joined_table, fieldNames) as cursor:
        
        # initialize variables for population summary formula
        numerator = 0
        denominator = 0
        
        # loop through each row, calculating the necessary values for each block
        # add those values to numerator and denominator to keep a running tally
        for row in cursor:
            
            # start and end values for trips for each block
            if row[0] == None: # convert null values to 0
                ti_start = 0
            else: ti_start = row[0]
            if row[1] == None:
                ti_end = 0
            else: ti_end = row[1]

            # start and end values for subgroup population for each block
            pi_start = row[2]
            pi_end = row[3]


            
            # calculate percent change in trips
            if ti_start == ti_end:
                delta_ti = 0 # to catch blocks where trips started and ended at 0
            else:
                
                
                try: 
                    delta_ti = (ti_end - ti_start) / ti_start 
                except:  # catch divide by zero errors
                    delta_ti = 1  # cap change at 100%, per Metro Transit's process
            delta_ti = min(delta_ti, 1)

           
            # do the same for changes in population
            if pi_start == pi_end:
                delta_pi = 0
            else:
                try:
                    delta_pi = (pi_end - pi_start) / pi_start
                except:
                    delta_pi = 1
            delta_pi = min(delta_pi, 1)

            # write block level service change value to table
            
            if delta_pi == -1:
                
                # A change in population of -100% would result in divide by 0 error in service level change formula. 
                # A value of -1 is appropriate since the population isn't being served in this block anymore
                row[4] = -1 
            else: 
                row[4] = (delta_ti + 1) / (delta_pi + 1) - 1 # rate of change in trips/person
                
            # update running totals for numerator and denominator
            numerator += pi_end * row[4] # weight change in trips/person by population
            denominator += pi_end
    
            # don't be an idiot and forget to actually save the changes
            cursor.updateRow(row)  

        # calculate total population weighted service level change across all blocks
        weighted_trip_change = numerator / denominator
        
        return population, weighted_trip_change

# Part 2: Establish parameters and run script

In [ ]:
# set parameters.  
# The workspace must be a gdb 
# The demography feature classes must be in a gdb, but they don't need to be the workspace
# The geometry of the demography feature classes for the start and end of the study period must be the same
# The demography feature classes must have a common field to join them together.

workspace = r'C:\Users\orms0027\Documents\GIS5572\ormst04\ormst04.gdb'
arcpy.env.workspace = workspace

start = {
    'demo': "demo_2010",                                      # feature class holding demographic variables
    'gtfs': r'C:\Users\orms0027\Documents\PA5234\gtfs_NOV09', # directory where all the gtfs data are stored
    'agency_ids': [0,1,2,4,5,6,7,8,9,10,11,12,13,14,15],      # from agency.txt, which agencies to analyze
    'service_ids': ['SEP09-Multi-Weekday-01'],                # from calendar.txt or calendar_dates.txt, the specific services to analyze
    'join_on': 'GISJOIN',                                     # field on which the two demography feature classes can be joined
}

end = {
    'demo': "demo_2019",
    'gtfs': r'C:\Users\orms0027\Documents\PA5234\gtfs_NOV19',
    'agency_ids': [0,1,2,4,5,6,7,8,9,10,11,12,13,14,15],
    'service_ids': ['AUG19-MVS-BUS-Weekday-01', 
                    'AUG19-MVS-NS-Weekday-01', 
                    'AUG19-RAIL-Weekday-01',
                    'AUG19-MVS-SUB-Weekday-01',
                    'AUG19-MVS-UM-Weekday-01'], # services were split in the later time period
    'join_on': 'GISJOIN',
}

# each subpopulation to be analyzed is a key in a dict, where the value is a list of field names
# the first item in the list is the field for the population count in the start demo fc,
# the second is the field for the population count in the end demo fc
populations = {
    'Total': ['H7X001', 'totpop'],
    'White alone': ['H7X002', 'raceandhispanicorigin_wht_cy'],
    'Black or African American alone' : ['H7X003', 'raceandhispanicorigin_black_cy'],
    'American Indian and Alaska Native alone': ['H7X004', 'raceandhispanicorigin_amerind_cy'],
    'Asian alone': ['H7X005', 'raceandhispanicorigin_asian_cy'],
    'Native Hawaiian and Other Pacific Islander alone': ['H7X006', 'raceandhispanicorigin_pacific_cy'],
    'Some Other Race alone': ['H7X007', 'raceandhispanicorigin_othrace_cy'], 
    'Two or More Races': ['H7X008', 'raceandhispanicorigin_race2up_cy'],
    'Hispanic': ['H73002', 'raceandhispanicorigin_hisppop_cy'],
    'Non-Hispanic White': ['H73005', 'raceandhispanicorigin_nhspwht_cy'],
    'Minority': ['MINPOP', 'MINPOP'],
    'People in Poverty': ['POVCOUNT', 'POVCOUNT'],
    'People not in Poverty': ['NONPOVCOUNT', 'NONPOVCOUNT']
}

# name of the output feature class holding the service level changes for each block for each subgroup
out_fc = 'service_change'

# full filepath of output csv with summary data for each subgroup
out_csv = r'C:\Users\orms0027\Documents\GIS5572\ormst04\service_change.csv'

In [ ]:
# run the analysis for scenario 3
df = main(start, end, populations, out_fc, out_csv)
df